
# Import neccessary modules
1 for matrix caculation, 2 for Deep Learning

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from module.input_parser_old import input_parser
from module import in_fun
from module.torch import BinaryClassifier


beacon_list = [f"beacon{idx + 1}" for idx in range(9)]

%matplotlib widget

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

# Load Data
python에서 csv파일을 읽어올때, 숫자만 있는것이 아닌 문자열이 포함된 데이터를 읽을 경우 느려지는 것 확인

In [3]:
data_1233_1 = input_parser(
    pd.read_csv(
        "./data/1233/note20/customscenario02_20210223173056.csv",
        header=None,
        low_memory=False,
    )
)
data_1233_2 = input_parser(
    pd.read_csv(
        "./data/1233/s20/customscenario02_20210223173056.csv",
        header=None,
        low_memory=False,
    )
)
data_1250 = input_parser(
    pd.read_csv(
        "./data/1250/customscenario01_20210223150144.csv", header=None, low_memory=False
    )
)
data_1251 = input_parser(
    pd.read_csv(
        "./data/1251/customscenario01_20201009224245.csv", header=None, low_memory=False
    )
) 
data_list = [data_1233_1, data_1233_2, data_1250, data_1251]

# Gather data
250 row(5초)의 데이터를 모아서, 한 row로 묶어서 처리하였다. RSSI는 250 row중 최댓값을 사용

In [4]:
gathered_list = []
for data in data_list:
    cur_gathered = np.zeros([np.shape(data)[0] // 250, 9])
    for row in range(np.shape(cur_gathered)[0]):
        cur_gathered[row] = np.max(data[row * 250 : row * 250 + 250, -9:], axis=0)
    cur_gathered += 100
    cur_gathered = cur_gathered / 100
    gathered_list.append(cur_gathered)
    print(np.shape(gathered_list[-1]))

(17, 9)
(17, 9)
(358, 9)
(368, 9)


In [5]:
gathered_1233 = np.concatenate((gathered_list[0], gathered_list[1]), axis=0)
gathered_1250 = gathered_list[2]
gathered_1251 = gathered_list[3]
room_data = [gathered_1233, gathered_1250, gathered_1251]
for room in room_data:
    print(np.shape(room))

(34, 9)
(358, 9)
(368, 9)


# "Same Room" Data
같은 방 데이터 끼리 매칭시키고, 그 둘 차의 절댓값으로 구성된 dim = 9의 input  
146 * 146 + 1495 * 1495 + 1535 * 1535 = 4612566

In [6]:
matched_same = []
for room in room_data:
    matched_same.append(in_fun.matcher(room, room))
    
same_diff = np.concatenate([same[0] for same in matched_same])
same_each = np.concatenate([same[1] for same in matched_same])
print(np.shape(same_diff))

(264744, 18)


# "Different Room" Data
다른 방 데이터 끼리 매칭시키고, 그 둘 차의 절댓값으로 구성된 dim = 9의 input  
146 * 1495 + 1495 * 1535 + 1535 * 146 = 2737205

In [7]:
matched_diff = []
matched_diff.append(in_fun.matcher(room_data[0], room_data[1]))
matched_diff.append(in_fun.matcher(room_data[1], room_data[2]))
matched_diff.append(in_fun.matcher(room_data[2], room_data[0]))

diff_diff = np.concatenate([diff[0] for diff in matched_diff])
diff_each = np.concatenate([diff[1] for diff in matched_diff])
print(diff_each[54554:54559])
print(np.shape(diff_each))

[[-1.   -1.   -1.   -1.   -1.   -1.   -1.    0.23  0.32 -1.   -1.   -1.
  -1.   -1.   -1.   -1.    0.23  0.32 -1.   -1.   -1.   -1.   -1.   -1.
  -1.    0.14  0.17 -1.   -1.   -1.   -1.   -1.   -1.   -1.    0.14  0.17]
 [-1.   -1.   -1.   -1.   -1.   -1.   -1.    0.23  0.32 -1.   -1.   -1.
  -1.   -1.   -1.   -1.    0.23  0.32 -1.   -1.   -1.   -1.   -1.   -1.
  -1.    0.16  0.22 -1.   -1.   -1.   -1.   -1.   -1.   -1.    0.16  0.22]
 [-1.   -1.   -1.   -1.   -1.   -1.   -1.    0.23  0.32 -1.   -1.   -1.
  -1.   -1.   -1.   -1.    0.23  0.32 -1.   -1.   -1.   -1.   -1.   -1.
  -1.    0.16  0.16 -1.   -1.   -1.   -1.   -1.   -1.   -1.    0.16  0.16]
 [-1.   -1.   -1.   -1.   -1.   -1.   -1.    0.23  0.32 -1.   -1.   -1.
  -1.   -1.   -1.   -1.    0.23  0.32 -1.   -1.   -1.   -1.   -1.   -1.
  -1.    0.15  0.22 -1.   -1.   -1.   -1.   -1.   -1.   -1.    0.15  0.22]
 [-1.   -1.   -1.   -1.   -1.   -1.   -1.    0.23  0.32 -1.   -1.   -1.
  -1.   -1.   -1.   -1.    0.23  0.32 -1.   -1.   -1

# Loading data for nn

In [8]:
from torch.utils.data import Dataset, TensorDataset
from torch.utils.data.dataset import random_split

x_tensor_diff = torch.from_numpy(np.concatenate((same_diff, diff_diff))).float()
x_tensor_each = torch.from_numpy(np.concatenate((same_each, diff_each))).float()

y = np.concatenate((np.zeros([len(same_each), 1]) + 1, np.zeros([len(diff_each), 1])))
y_tensor = torch.from_numpy(y).float()

dataset_diff = TensorDataset(x_tensor_diff, y_tensor)
dataset_each = TensorDataset(x_tensor_each, y_tensor)

train_len_diff = len(dataset_diff) // 5 * 4
train_len_each = len(dataset_each) // 5 * 4
val_len_diff = len(dataset_diff) - train_len_diff
val_len_each  = len(dataset_each) - train_len_each

train_dataset_diff, val_dataset_diff = random_split(dataset_diff, [train_len_diff, val_len_diff])
train_dataset_each, val_dataset_each = random_split(dataset_each, [train_len_each, val_len_each])

train_loader_diff = DataLoader(dataset=train_dataset_diff, batch_size=32, shuffle=True)
train_loader_each = DataLoader(dataset=train_dataset_each, batch_size=32, shuffle=True)
val_loader_diff = DataLoader(dataset = val_dataset_diff, batch_size = val_len_diff)
val_loader_each = DataLoader(dataset = val_dataset_each, batch_size = val_len_each)

# Layer
nn 생성

In [9]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
print(device)

cpu


In [10]:
net_diff = BinaryClassifier(input_size = 18)
net_each = BinaryClassifier(input_size = 36)
net_diff.to(device)
net_each.to(device)
optimizer_diff = optim.SGD(net_diff.parameters(), lr=3e-3)
optimizer_each = optim.SGD(net_each.parameters(), lr=3e-3)
criterion = nn.BCEWithLogitsLoss()

비교분석을 위한 수치  
모든 데이터에 true라고 응답해도, 정확도가 62%는 나온다.

In [11]:
print(len(same_each) / (len(same_each) + len(diff_each)))

0.6285887950765958


In [12]:
num_epoch = 100
for epoch in range(num_epoch):
    for batch_idx, data in enumerate(train_loader_diff):
        x_train, y_train = data[0].to(device), data[1].to(device)
        hypothesis = net_diff(x_train)
        cost = criterion(hypothesis, y_train)
        # cost로 H(x) 계산
        optimizer_diff.zero_grad()
        cost.backward()
        optimizer_diff.step()

    with torch.no_grad():
        for data in val_loader_diff:
            x_val, y_val = data
            hypothesis = net_diff(x_val)
            prediction = hypothesis >= torch.FloatTensor([0.5])
            correct_prediction = prediction.float() == y_val  # 실제값과 일치하는 경우만 True로 간주
            val_accuarcy = correct_prediction.sum().item() / len(correct_prediction)  # 정확도를 계산
            print(f"epoch: {epoch}, Val Accuarcy : {val_accuarcy * 100}%")


/opt/conda/lib/python3.7/site-packages/torch/autograd/__init__.py:132: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370156314/work/c10/cuda/CUDAFunctions.cpp:100.)
  allow_unreachable=True)  # allow_unreachable flag


epoch: 0, Val Accuarcy : 99.20698988555962%
epoch: 1, Val Accuarcy : 99.38149959637211%
epoch: 2, Val Accuarcy : 99.48596799468162%
epoch: 3, Val Accuarcy : 99.4990265444703%
epoch: 4, Val Accuarcy : 99.54295075739589%
epoch: 5, Val Accuarcy : 99.53107934849707%
epoch: 6, Val Accuarcy : 99.60586922455957%
epoch: 7, Val Accuarcy : 99.61536635167862%
epoch: 8, Val Accuarcy : 99.61536635167862%
epoch: 9, Val Accuarcy : 99.61536635167862%
epoch: 10, Val Accuarcy : 99.62367633790778%
epoch: 11, Val Accuarcy : 99.62605061968755%
epoch: 12, Val Accuarcy : 99.62605061968755%
epoch: 13, Val Accuarcy : 99.63079918324706%
epoch: 14, Val Accuarcy : 99.63079918324706%
epoch: 15, Val Accuarcy : 99.63910916947624%
epoch: 16, Val Accuarcy : 99.63792202858636%
epoch: 17, Val Accuarcy : 99.63317346502683%
epoch: 18, Val Accuarcy : 99.64029631036611%
epoch: 19, Val Accuarcy : 99.63910916947624%
epoch: 20, Val Accuarcy : 99.64029631036611%
epoch: 21, Val Accuarcy : 99.64979343748516%
epoch: 22, Val Accuar

In [13]:
num_epoch = 100
for epoch in range(num_epoch):
    for batch_idx, data in enumerate(train_loader_each):
        x_train, y_train = data[0].to(device), data[1].to(device)
        hypothesis = net_each(x_train)
        cost = criterion(hypothesis, y_train)
        # cost로 H(x) 계산
        optimizer_each.zero_grad()
        cost.backward()
        optimizer_each.step()

    with torch.no_grad():
        for data in val_loader_each:
            x_val, y_val = data
            hypothesis = net_each(x_val)
            prediction = hypothesis >= torch.FloatTensor([0.5])
            correct_prediction = prediction.float() == y_val  # 실제값과 일치하는 경우만 True로 간주
            val_accuarcy = correct_prediction.sum().item() / len(correct_prediction)  # 정확도를 계산
            print(f"epoch: {epoch}, Val Accuarcy : {val_accuarcy * 100}%")


epoch: 0, Val Accuarcy : 97.51768839925923%
epoch: 1, Val Accuarcy : 99.67947195973218%
epoch: 2, Val Accuarcy : 99.72102189087802%
epoch: 3, Val Accuarcy : 99.83617455719644%
epoch: 4, Val Accuarcy : 99.8634787976637%
epoch: 5, Val Accuarcy : 99.86110451588395%
epoch: 6, Val Accuarcy : 99.84685882520537%
epoch: 7, Val Accuarcy : 99.89671874258038%
epoch: 8, Val Accuarcy : 99.89197017902084%
epoch: 9, Val Accuarcy : 99.83973597986609%
epoch: 10, Val Accuarcy : 99.8694145021131%
epoch: 11, Val Accuarcy : 99.86704022033335%
epoch: 12, Val Accuarcy : 99.8955316016905%
epoch: 13, Val Accuarcy : 99.91215157414882%
epoch: 14, Val Accuarcy : 99.90384158791966%
epoch: 15, Val Accuarcy : 99.8824730519018%
epoch: 16, Val Accuarcy : 99.49427798091077%
epoch: 17, Val Accuarcy : 99.88009877012203%
epoch: 18, Val Accuarcy : 99.91096443325894%
epoch: 19, Val Accuarcy : 99.9133387150387%
epoch: 20, Val Accuarcy : 99.91690013770834%
epoch: 21, Val Accuarcy : 99.91571299681846%
epoch: 22, Val Accuarcy :

# 개선 계획
학습 과정에서 accuracy가 높아지지 않은 것으로, 제대로 학습되지 않는 상태인것으로 확인  
input에 대한 조절로 결과를 얻고자한다
1. RSSI 값 연산 시 재정의:   
현재 해당 비컨에 해당하는 RSSI가 없으면, 0으로 넣어주고 비교하고있다.  
그결과, null데이터와 -90dB의 차이가, null데이터와 -10dB의 차이보다 더 큰 상태이다.  
null값에 대해 -100을 넣어주고 연산하는 방식으로 오류를 줄여야 한다.  
2. Input 범위 재설정:  
1의 적용 이후, 실제 input으로 들어갈 값들은 -100 ~ -1 사이의 값이다.  
input에 100을 더한 후 100으로 나눠주어, input들이 0 ~ 1 사이의 범위에 있도록 normalize 해줄 계획이다
3. Input Data 추가:  
현재는 두 데이터간의 '차의 절댓값' 만을 넣어주고 있어서, 데이터 dimension이 9이하인 상태이다. (적은 편)  
차만을 사용하는 과정에서, 각 데이터의 RF measurement들이 버려지게 된다.  
4. Input의 다원화:  
현재 사용하고 있지 않은 기압값등을 어떻게 활용할지 고려한다.

In [14]:
torch.save(net_diff.state_dict(), "./model/diff")
torch.save(net_each.state_dict(), "./model/each")